In [5]:

import sys
sys.path.append('..\\helpers')

import cv2
import numpy as np
from preprocessing import Preprocessor

preprocessor = Preprocessor()


# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def upsample(image):
    height, width = image.shape[:2]
    new_height = int(height * 4)
    new_width = int(width * 4)
    
    return cv2.resize(image.copy(), (new_width, new_height), interpolation=cv2.INTER_CUBIC)

def binarize(image):
    threshold = 128
    _, binarized_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return binarized_image

def show_image(labe, image):    
    cv2.imshow(labe, cv2.resize(image, (image.shape[1]//3, image.shape[0]//3)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def convert_to_8bit(image_16bit):
    if not isinstance(image_16bit, np.ndarray) or image_16bit.dtype != np.uint16:
        raise ValueError("Input must be a 16-bit numpy array")

    # Scale the 16-bit image to the range of 0-255
    image_8bit = (image_16bit / np.max(image_16bit) * 255).astype(np.uint8)

    return image_8bit


In [42]:
image_name = '22614379_d065adcb9905b973_MG_L_CC_ANON' # without .tif
mms_scale  = 5

gt_image   = cv2.imread(f'../dataset/processed/groundtruth/{image_name}.tif', cv2.IMREAD_GRAYSCALE)
segm_image = cv2.imread(f'../dataset/output/segmentation/{image_name}_{mms_scale}.tif', cv2.IMREAD_GRAYSCALE)
raw_image  = cv2.imread(f'../dataset/processed/cropped_raw/{image_name}.tif', cv2.IMREAD_UNCHANGED)

gt_image   = binarize(upsample(gt_image))
segm_image = binarize(upsample(segm_image))

raw_image  = convert_to_8bit(raw_image)
raw_image  = cv2.cvtColor(raw_image, cv2.COLOR_GRAY2BGR)

# get the contours for both gt and seg
gt_contours, _  = cv2.findContours(gt_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
seg_contours, _ = cv2.findContours(segm_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the raw image
color_gt   = (0, 0, 0)   # black for gt
color_seg  = (0, 0, 255) # red for seg
cv2.drawContours(raw_image, gt_contours, -1, color_gt, thickness=4)
cv2.drawContours(raw_image, seg_contours, -1, color_seg, thickness=4)


# Display DICE score for the segmentation
for gt_cnt in gt_contours:
    max_dice = 0

    gt_candidate = np.zeros_like(gt_image)
    cv2.drawContours(gt_candidate, [gt_cnt], 0, (255, 255, 255), cv2.FILLED)                

    for seg_cnt in seg_contours:        
        mask_candidate = np.zeros_like(segm_image)
        cv2.drawContours(mask_candidate, [seg_cnt], 0, (255, 255, 255), cv2.FILLED)                

        intersection = cv2.bitwise_and(mask_candidate, gt_candidate)
        union = cv2.bitwise_or(mask_candidate, gt_candidate)

        intersection_pixels = cv2.countNonZero(intersection)
        union_pixels = cv2.countNonZero(union)

        dice_score = (2 * intersection_pixels) / (intersection_pixels + union_pixels)

        if dice_score > max_dice:
            max_dice = dice_score

            x, y, w, h = cv2.boundingRect(seg_cnt) 
    cv2.putText(raw_image, "DICE SCORE: " + str(round(max_dice, 4)), (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, color_seg, 4)


# show_image("raw_image", raw_image)

preprocessor.save_image(raw_image, '../report/report_images/visualize_segmentation/', f'{image_name}_{mms_scale}.tif')





Image saved successfully to: ../report/report_images/visualize_segmentation/22614379_d065adcb9905b973_MG_L_CC_ANON_5.tif
